<a href="https://colab.research.google.com/github/sera0911/asia-ai-study/blob/main/pandas_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###pandas 시계열  
####시계열 데이터  
- 시계열 자료는 인덱스가 날짜 혹은 시간인 데이터
- 시간 내에서 특정 순간의 타임스탬프
- 2017년 1월, 2019년 전체 같은 고정된 기간
- 시작과 끝 타임스탬프로 표시되는 시간 간격 (기간은 시간 간격의 특수한 경우)
- 실험 혹은 경과 시간, 각 타임스탬프는 특정 시작 시간에 상대적인 시간의 특정값
- 시계열은 고정빈도로 표현 – 데이터가 존재하는 지점이 15초마다, 5분마다, 한달에 한번 같은 특정 규칙에 따라 고정 간격을 가지게 된다.
- 고정된 단위나 시간 혹은 단위들 간의 간격으로 존재하지 않고 불규칙적인 모습으로 표현될 수도 있다
- 판다스에서 시계열 자료를 생성하려면 인덱스를 DatetimeIndex 자료형으로 만들어야 한다.
- DatetimeIndex는 특정한 순간에 기록된 타임스탬프(timestamp) 형식의 시계열 자료를 다루기 위한 인덱스이다.
- 타임스탬프 인덱스의 라벨값이 반드시 일정한 간격일 필요는 없다.

####파이썬 표준 라이브러리 – 날짜, 시간 자료형 , 도구
- 날짜와 시간을 위한 자료형과 달력 관련 기능을 제공
- datetime, time, calendar
- datetime은 날짜와 시간을 모두 저장하며 마이크로초까지 지원한다.

In [1]:
from datetime import datetime
now = datetime.now()
print(now)
print(type(now))
print(now.year)
print(now.month)
print(now.day)

2021-04-05 06:58:46.762142
<class 'datetime.datetime'>
2021
4
5


- datetime.timedelta : 두 datetime 객체 간의 시간적인 차이를 표현

In [2]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)  #괄호 앞에서부터 년, 월, 일, 시간, 분순, 초까지 적을 수 있다
print(delta)
print(type(delta))
print(delta.days)
print(delta.seconds)

926 days, 15:45:00
<class 'datetime.timedelta'>
926
56700


- timedelta 를 더하거나 빼면 그만큼의 시간이 datetime 객체에 적용되어 새로운 객체를 만들 수 있다

In [3]:
from datetime import timedelta
start = datetime(2011, 1, 7)    
print(start)
print(start + timedelta(12))  #timedelta는 day를 기준으로 봄
print(start - 2 * timedelta(12))

2011-01-07 00:00:00
2011-01-19 00:00:00
2010-12-14 00:00:00


- datetime 객체와 pandas의 Timestamp 객체는 str(), strftime()에 포맷규칙을 넘겨서 문자열로 표현 할 수 있다

In [4]:
stamp = datetime(2011, 1, 3)
print(str(stamp))
print(stamp.strftime('%Y-%m-%d'))

2011-01-03 00:00:00
2011-01-03


- datetime.strptime : 알려진 형식의 날짜를 파싱하는 최적의 방법

In [9]:
value = '2011-01-03'
print(type(value))
value2 = print(datetime.strptime(value, '%Y-%m-%d'))
print(value2)
print(type(value2))
datestrs = ['7/6/2011', '8/6/2011']
d1 = [datetime.strptime(x, '%m/%d/%Y') for x in datestrs]
print(d1)

<class 'str'>
2011-01-03 00:00:00
None
<class 'NoneType'>
[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]


- 흔히 쓰는 날짜 형식의 포맷 규칙은 서드파티 패키지인 dateutil이 포함된 parser.parse()를 사용하면 편리   
- 국제 로케일의 경우 날짜가 월 앞에 오는 경우 dayfirst=True를 넘겨준다  

- strptime, parse 모두 datetime객체로 변환해줌

In [10]:
from dateutil.parser import parse
print(type('2011-01-03'))   #문자열
print(parse('2011-01-03'))
print(type(parse('2011-01-03')))   #datetime객체로 변환 

print(parse('Jan 31, 1997 10:45 PM'))
print(parse('6/12/2011', dayfirst=True))

<class 'str'>
2011-01-03 00:00:00
<class 'datetime.datetime'>
1997-01-31 22:45:00
2011-12-06 00:00:00


- pandas는 DateFrame의 컬럼이나 축 색인으로 날짜가 담긴 배열을 사용한다
- to_datetime() - 많은 종류의 날짜 표현을 처리, format 매개변수에 날짜와 시간 포맷을 지정
- errors 매개변수 - 오류 처리, 누락된 값(None, 빈 문자열 등)으로 간주되어야 할 값도 처리해준다 (NaT는 pandas에서 누락된 타임스탬프 데이터를 나타낸다)

In [13]:
import pandas as pd
import numpy as np

datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']
d1 = pd.to_datetime(datestrs)  #datetime객체로 변환
print(d1)
print(type(d1))
idx = pd.to_datetime(datestrs + [None])   #datestrs에 [none]이 추가되었는데 NaT로 표현됨
print(idx)
print(idx[2])
print(pd.isnull(idx))   #null값을 체크 앞에 두가지는 null이 아니라서 false 뒤는 null이라 true

date_strings = np.array(['03-04-2005 11:35 PM',
'23-05-2010 12:01 AM',
'04-09-2009 09:09 PM']) # 문자열
# Timestamp 객체로 변환
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]  #리스트컴프리헨션
[pd.to_datetime(date, format="%d-%m-%Y %I:%M %p", errors="ignore") for date in date_strings]
pd.to_datetime(date_strings)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)
NaT
[False False  True]


DatetimeIndex(['2005-03-04 23:35:00', '2010-05-23 00:01:00',
               '2009-04-09 21:09:00'],
              dtype='datetime64[ns]', freq=None)

- 파이썬 문자열이나 datetime 객체로 표현되는 타임스탬프로 색인된 Series

In [16]:
from datetime import datetime
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
datetime(2011, 1, 7), datetime(2011, 1, 8),
datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index=dates)  #randn 0보다 작거나 크면서 1과 가까운 난수
display(ts ) #datetime 객체는 DatetimeIndex에 들어 있으며 ts 변수의 타입은 TimeSeries

print(ts.index ) #서로 다르게 색인된 시계열 객체 간의 산술 연산은 자동으로 날짜에 맞춰진다

print(ts + ts[::2])   #ts +ts를 할 때 난수값은 2단위로 출력됨rint(ts + ts[::2])   #ts +ts를 할 때 난수값은 2단위로 출력됨

print(ts.index.dtype) # DatetimeIndex의 스칼라 값은 pandas의 Timestamp 객체
stamp = ts.index[0]
print(stamp)

print(ts.index.dtype) # DatetimeIndex의 스칼라 값은 pandas의 Timestamp 객체
stamp = ts.index[0]
print(stamp)

2011-01-02    0.555662
2011-01-05   -1.021371
2011-01-07   -1.428447
2011-01-08   -1.021198
2011-01-10   -0.344437
2011-01-12    2.032549
dtype: float64

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)
2011-01-02    1.111324
2011-01-05         NaN
2011-01-07   -2.856894
2011-01-08         NaN
2011-01-10   -0.688875
2011-01-12         NaN
dtype: float64
datetime64[ns]
2011-01-02 00:00:00


- 시계열은 라벨에 기반해서 데이터를 선택하고 인덱싱할 때 pandas.Series와 동일하게 동작한다

In [19]:
from datetime import datetime

#해석할 수 있는 날짜를 문자열로 넘겨서 편리하게 사용할 수 있다.
print( ts['1/10/2011'] )
print(ts['20110110'])
#긴 시계열에서는 년을 넘기거나, 월만 넘겨서 데이터의 일부 구간만 선택할 수도 있다.
longer_ts = pd.Series(np.random.randn(1000),
index=pd.date_range('1/1/2000', periods=1000))

print(longer_ts)
print(longer_ts['2001'])
print(longer_ts['2001-05'])
print(ts[datetime(2011, 1, 7):])
#시계열 데이터는 연대순으로 정렬되기 때문에 범위를 지정하기 위해 시계열에 포함하지 않고 
#타임스탬프를 이용해서 Series를 나눌 수 있다
print(ts)
print(ts['1/6/2011':'1/11/2011'])

-0.34443740884187735
-0.34443740884187735
2000-01-01    0.789652
2000-01-02    1.027059
2000-01-03   -0.511049
2000-01-04    1.129649
2000-01-05    0.365506
                ...   
2002-09-22   -0.210771
2002-09-23    0.909179
2002-09-24   -0.099478
2002-09-25    1.346553
2002-09-26    0.497708
Freq: D, Length: 1000, dtype: float64
2001-01-01    0.603129
2001-01-02    0.659621
2001-01-03   -1.000177
2001-01-04    1.005032
2001-01-05   -0.168542
                ...   
2001-12-27   -0.105301
2001-12-28   -0.258748
2001-12-29   -1.821075
2001-12-30   -0.667880
2001-12-31   -0.632471
Freq: D, Length: 365, dtype: float64
2001-05-01   -0.468367
2001-05-02    0.557140
2001-05-03    0.252296
2001-05-04    0.401868
2001-05-05   -0.203239
2001-05-06   -0.411643
2001-05-07    1.385024
2001-05-08    0.671476
2001-05-09   -0.007249
2001-05-10    0.243270
2001-05-11   -0.820988
2001-05-12    2.025688
2001-05-13   -1.479152
2001-05-14   -1.840209
2001-05-15    0.392574
2001-05-16   -0.077201
2001-05-1

In [20]:
#truncate()는 TimeSeries를 두 개의 날짜로 나눈다.
print(ts.truncate(after='1/9/2011'))   #after는 잘라내는 범위 1월9일 이전으로! 라는 뜻

#periods 간격 freq는 수요일마다! =2000/1/1부터 100일간 수요일을 출력
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')   
print(dates)
long_df = pd.DataFrame(np.random.randn(100, 4),    #100행4열로 만듬 
index=dates,
columns=['Colorado', 'Texas',
'New York', 'Ohio'])

print(long_df)
print(long_df.loc['5-2001'])

2011-01-02    0.555662
2011-01-05   -1.021371
2011-01-07   -1.428447
2011-01-08   -1.021198
dtype: float64
DatetimeIndex(['2000-01-05', '2000-01-12', '2000-01-19', '2000-01-26',
               '2000-02-02', '2000-02-09', '2000-02-16', '2000-02-23',
               '2000-03-01', '2000-03-08', '2000-03-15', '2000-03-22',
               '2000-03-29', '2000-04-05', '2000-04-12', '2000-04-19',
               '2000-04-26', '2000-05-03', '2000-05-10', '2000-05-17',
               '2000-05-24', '2000-05-31', '2000-06-07', '2000-06-14',
               '2000-06-21', '2000-06-28', '2000-07-05', '2000-07-12',
               '2000-07-19', '2000-07-26', '2000-08-02', '2000-08-09',
               '2000-08-16', '2000-08-23', '2000-08-30', '2000-09-06',
               '2000-09-13', '2000-09-20', '2000-09-27', '2000-10-04',
               '2000-10-11', '2000-10-18', '2000-10-25', '2000-11-01',
               '2000-11-08', '2000-11-15', '2000-11-22', '2000-11-29',
               '2000-12-06', '2000-12-13'

In [21]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)   #np.arange은 배열을 나타냄
print(dup_ts)

#is_unique 속성으로 색인 유일 여부 확인
print( dup_ts.index.is_unique )  #유니크한가를 물어봤을 때 중복이 있으니 false
print(dup_ts['1/3/2000'] ) # not duplicated  = 중복이아니다 4번째위치값이니 4
print(dup_ts['1/2/2000'] ) # duplicated = 중복이다 위치를 알려줌

# 유일하지 않은 타임스탬프를 가지는 데이터 집계
#groupby에 level=0(단일 단계 인덱싱)을 넘긴다

grouped = dup_ts.groupby(level=0)
grouped.mean()  #갯수 평균
grouped.count()  #갯수 카운트

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64
False
4
2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64


2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

- pandas는 시계열을 불규칙적인 것으로 간주 (고정된 빈도를 갖지 않는다)
- pandas에는 리샘플릴링, 표준 시계열 빈도 모음, 빈도 추론, 고정된 빈도의 날짜 범위를 위한 도구 제공
- pandas.date_range를 사용하면 특정 빈도에 따라 지정한 길이만큼의 DatetimeIndex를 생성
- pandas.date_range는 기본적으로 시작 시간이나 종료 시간의 타임 스탬프를 보존한다
- 자정에 맞추어 타임스탬프를 정규화하고 싶을 때 normalize 옵션을 사용

In [22]:
display(ts)
resampler = ts.resample('D')   #freq=<Day> 간격을 일(day) 단위로 지정시킨 것것
print(resampler)
#date_range는 일별 타임스탬프를 생성
index = pd.date_range('2012-04-01', '2012-06-01')
print(index)

print(pd.date_range(start='2012-04-01', periods=20))  #start를 기준으로 생성
print(pd.date_range(end='2012-06-01', periods=20))  #end를 기준으로 생성

#빈도값 BM BusinessMonthEnd (주말이 아닌 평일 중에서 각 달의 마지막 날)
print(pd.date_range('2000-01-01', '2000-12-01', freq='BM') )

2011-01-02    0.555662
2011-01-05   -1.021371
2011-01-07   -1.428447
2011-01-08   -1.021198
2011-01-10   -0.344437
2011-01-12    2.032549
dtype: float64

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, origin=start_day]
DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', 

- pandas에서 빈도는 기본 빈도와 배수의 조합으로 이루어진다.

In [29]:
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
print(hour)

four_hours = Hour(4)  #Hour에 4를 넣으면 <4 * Hours>
print(four_hours)

print(pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h'))  #freq='4h 는 4시간 간격으로 출력
print(pd.date_range('2000-01-01', '2000-01-03 23:59', freq=four_hours)) #위에 값과 동일, 위에서 4시간씩 *하기때문에
print(pd.date_range('2000-01-01', '2000-01-03 23:59', freq=hour)) #1시간단위로 출력

#여러 오프셋을 덧셈으로 합칠 수 있다
print(Hour(2) + Minute(30))   #2시간30분씩  <150 * Minutes>

print(pd.date_range('2000-01-01', periods=10, freq='1h30min'))  #10개의 기간을 1시간 30분단위로 출력

rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI') #WOM-3FRI = 각 달의 3번째 금요일
list(rng)

<Hour>
<4 * Hours>
DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')
DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

#### pandas 날짜 범위, 빈도, 이동
- shift는 데이터를 시간 축에서 앞이나 뒤로 이동하는 것을 의미
- shift() - Series와 DataFrame은 색인은 변경하지 않고 데이터를 앞이나 뒤로 느슨한 시프트를 수행

In [30]:
ts = pd.Series(np.random.randn(4),
index=pd.date_range('1/1/2000', periods=4, freq='M'))

print(ts)
print(ts.shift(2))  #shift = 날짜 범위 이동
print(ts.shift(-2))
print(ts.shift(2, freq='M'))  #월이 이동
print(ts.shift(3, freq='D'))  #일자로 이동
print(ts.shift(1, freq='90T'))  #90분

2000-01-31   -0.454547
2000-02-29   -1.985071
2000-03-31    1.067279
2000-04-30    0.516663
Freq: M, dtype: float64
2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.454547
2000-04-30   -1.985071
Freq: M, dtype: float64
2000-01-31    1.067279
2000-02-29    0.516663
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64
2000-03-31   -0.454547
2000-04-30   -1.985071
2000-05-31    1.067279
2000-06-30    0.516663
Freq: M, dtype: float64
2000-02-03   -0.454547
2000-03-03   -1.985071
2000-04-03    1.067279
2000-05-03    0.516663
dtype: float64
2000-01-31 01:30:00   -0.454547
2000-02-29 01:30:00   -1.985071
2000-03-31 01:30:00    1.067279
2000-04-30 01:30:00    0.516663
dtype: float64


#### resample 연산
- resample() - 시간 간격을 재조정하는 리샘플링(resampling)이 가능하다.
- 시간 구간이 작아지면 데이터 양이 증가한다고 해서 업-샘플링(up-sampling)이라 하고 시간 구간이 커지면 데이터
양이 감소한다고 해서 다운-샘플링(down-sampling)이라 부른다.
- 날짜가 아닌 시/분 단위에서는 구간위 왼쪽 한계값(가장 빠른 값)은 포함하고 오른쪽 한계값(가장 늦은 값)은 포함
하지 않는다.
- 왼쪽이 아니라 오른쪽 한계값을 구간에 포함하려면 closed="right" 인수를 사용한다.
- ohlc() - 구간의 시고저종(open, high, low, close)값을 구한다.

In [33]:
ts = pd.Series(np.random.randn(100), index=pd.date_range( "2018-1-1", periods=100, freq="D"))
#1월1일부터 일자로 구간을 정해 100개
print(ts.tail(20)) #끝에 20개 출력
print(ts.resample('W').mean())   #W=일요일
print(ts.resample('M').first())  #M=달의 마지막날

2018-03-22   -0.417399
2018-03-23    0.234706
2018-03-24   -0.377653
2018-03-25   -1.164829
2018-03-26   -0.755154
2018-03-27   -0.841421
2018-03-28   -1.066766
2018-03-29    0.947633
2018-03-30   -2.120330
2018-03-31    0.039432
2018-04-01   -0.802918
2018-04-02   -0.911942
2018-04-03   -1.203855
2018-04-04    0.433098
2018-04-05    0.875605
2018-04-06    1.309833
2018-04-07   -0.000067
2018-04-08    0.521812
2018-04-09   -2.111367
2018-04-10    0.694617
Freq: D, dtype: float64
2018-01-07    0.185053
2018-01-14    0.146386
2018-01-21   -0.156095
2018-01-28    0.391992
2018-02-04    0.191336
2018-02-11    0.523066
2018-02-18   -0.186828
2018-02-25    0.462134
2018-03-04   -0.047259
2018-03-11    0.522293
2018-03-18    0.128602
2018-03-25   -0.358600
2018-04-01   -0.657075
2018-04-08    0.146355
2018-04-15   -0.708375
Freq: W-SUN, dtype: float64
2018-01-31    0.351928
2018-02-28   -1.044910
2018-03-31    0.144717
2018-04-30   -0.802918
Freq: M, dtype: float64


In [35]:
ts = pd.Series(np.random.randn(60), index=pd.date_range( "2018-1-1", periods=60, freq="T"))
#2018/1/1부터 분을 60개 시리즈로
print(ts.head(20))  #앞20개
print(ts.resample('10T').sum())  #10분씩 합
print(ts.resample('10T', closed="right").sum())  #한계값을 포함해서 10분씩 합
print(ts.resample('5T').ohlc())  #ohlc() - 구간의 시고저종(open, high, low, close)값을 구한다.

2018-01-01 00:00:00   -1.309365
2018-01-01 00:01:00    0.996381
2018-01-01 00:02:00   -0.419214
2018-01-01 00:03:00   -0.511117
2018-01-01 00:04:00   -1.617696
2018-01-01 00:05:00   -0.535703
2018-01-01 00:06:00   -0.232352
2018-01-01 00:07:00    0.552810
2018-01-01 00:08:00   -0.149426
2018-01-01 00:09:00    0.570646
2018-01-01 00:10:00    0.042114
2018-01-01 00:11:00    1.213302
2018-01-01 00:12:00    0.704013
2018-01-01 00:13:00   -1.949418
2018-01-01 00:14:00   -1.056339
2018-01-01 00:15:00   -0.137770
2018-01-01 00:16:00   -1.858814
2018-01-01 00:17:00    0.772826
2018-01-01 00:18:00    1.097694
2018-01-01 00:19:00   -0.100471
Freq: T, dtype: float64
2018-01-01 00:00:00   -2.655035
2018-01-01 00:10:00   -1.272863
2018-01-01 00:20:00   -2.328691
2018-01-01 00:30:00   -1.507758
2018-01-01 00:40:00    2.496700
2018-01-01 00:50:00    2.674656
Freq: 10T, dtype: float64
2017-12-31 23:50:00   -1.309365
2018-01-01 00:00:00   -1.303556
2018-01-01 00:10:00   -1.290581
2018-01-01 00:20:00   

- 업-샘플링의 경우에는 실제로 존재하지 않는 데이터를 만들어야 한다.
- 업-샘플링의 경우, 앞에서 나온 데이터를 뒤에서 그대로 쓰는 forward filling 방식과 뒤에서 나올 데이터를 앞에서
미리 쓰는 backward filling 방식을 사용할 수 있다.
- ffill(), bfill()
- datetime 자료형 Series의 dt 접근자 - datetime 자료형이 가진 몇가지 유용한 속성과 메서드를 사용할 수 있다.

In [36]:
print(ts.resample('30s').ffill().head(20))
print(ts.resample('30s').bfill().head(20))

s = pd.Series(pd.date_range("2020-12-25", periods=100, freq="D"))
print(s)
print(s.dt.year)
print(s.dt.weekday)  #여기서 0,1,2,3,4,5,6은 월화수목금토일을 뜻함
#strftime() - 문자열을 만듬
print(s.dt.strftime("%Y년 %m월 %d일"))

2018-01-01 00:00:00   -1.309365
2018-01-01 00:00:30   -1.309365
2018-01-01 00:01:00    0.996381
2018-01-01 00:01:30    0.996381
2018-01-01 00:02:00   -0.419214
2018-01-01 00:02:30   -0.419214
2018-01-01 00:03:00   -0.511117
2018-01-01 00:03:30   -0.511117
2018-01-01 00:04:00   -1.617696
2018-01-01 00:04:30   -1.617696
2018-01-01 00:05:00   -0.535703
2018-01-01 00:05:30   -0.535703
2018-01-01 00:06:00   -0.232352
2018-01-01 00:06:30   -0.232352
2018-01-01 00:07:00    0.552810
2018-01-01 00:07:30    0.552810
2018-01-01 00:08:00   -0.149426
2018-01-01 00:08:30   -0.149426
2018-01-01 00:09:00    0.570646
2018-01-01 00:09:30    0.570646
Freq: 30S, dtype: float64
2018-01-01 00:00:00   -1.309365
2018-01-01 00:00:30    0.996381
2018-01-01 00:01:00    0.996381
2018-01-01 00:01:30   -0.419214
2018-01-01 00:02:00   -0.419214
2018-01-01 00:02:30   -0.511117
2018-01-01 00:03:00   -0.511117
2018-01-01 00:03:30   -1.617696
2018-01-01 00:04:00   -1.617696
2018-01-01 00:04:30   -0.535703
2018-01-01 00:

#### 시계열 데이터에서 누락된 값 처리
- 시계열 데이터에서는 보간(interpolation)방법을 사용하여 누락된 값으로 생긴 간격을 채움.
- 보간 - 누락된 값의 양쪽 경계를 잇는 직선이나 곡선을 사용하여 적절한 값을 예측함으로써 비어 있는 간극을 채우
는 기법
- 시간 간격이 일정하고, 데이터가 노이즈로 인한 변동이 심하지 않고 누락된 값으로 인한 빈 간극이 작을 때 보간
방법이 유용
- 두 포인트 사이의 직선이 비선형이라고 가정하면 interpolate의 method 매개변수를 사용해 다른 보간 방법을 지정
할 수 있습니다.
- 누락된 값의 간격이 커서 전체 간격을 보간하는 것이 좋지 않을 때는 limit 매개변수를 사용하여 보간 값의 개수를 제한하고 limit_direction 매개변수에서 마지막 데이터로 앞쪽 방향으로 보간할지 그 반대로 할지 지정할 수 있습
니다.

In [40]:
time_index = pd.date_range("01/01/2010", periods=5, freq="M")
dataframe = pd.DataFrame(index=time_index) # 데이터프레임을 만들고 인덱스를 지정

dataframe["Sales"] = [1.0,2.0,np.nan,np.nan,5.0]   # 누락된 값이 있는 특성을 만듬
dataframe.interpolate()   # 누락된 값을 보간

print(dataframe)
#print(dataframe.ffill())
#print(dataframe.bfill())

dataframe.ffill() # 앞쪽으로 채우기(Forward-fill)
dataframe.bfill() # 뒤쪽으로 채우기(Back-fill)
dataframe.interpolate(method="quadratic") # 비선형의 경우 보간 방법 변경
dataframe.interpolate(limit=1, limit_direction="forward") #보간 방향 지정 , limit=1은 1개만 채우라는 것 forward는 진행방향으로

            Sales
2010-01-31    1.0
2010-02-28    2.0
2010-03-31    NaN
2010-04-30    NaN
2010-05-31    5.0


,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,NaN
2010-05-31,5.0


#### 이동 시간 윈도 사용
- 시계열 데이터에서 일정 시간 간격으로 통계를 계산할 수 있습니다.
- 예] 세 달을 시간 윈도로 이동 평균을 계산 : mean(January, February, March) , mean(February, March, April), mean(March, April, May ) 세 달 크기의 시간 윈도가 각 단계마다 윈도의 평균을 계산하면서 샘플 위를 이동합니다.
- rolling() - window 매개변수에서 윈도 크기를 지정합니다.
- 시간 윈도의 max(), mean(), count(), corr()와 같은 통계를 간편하게 계산할 수 있습니다.
- ewm() - 최근 항목에 높은 가중치를 두지만 전체 기간에 대한 통계를 계산하는 지수 이동 윈도

In [43]:
import pandas as pd

# datetime을 만듭니다.
time_index = pd.date_range("01/01/2010", periods=5, freq="M")
# 데이터프레임을 만들고 인덱스를 설정합니다.
dataframe = pd.DataFrame(index=time_index)

dataframe["Stock_Price"] = [1,2,3,4,5] # 특성을 만듭니다.
print(dataframe)
print(dataframe.rolling(window=2).mean()) # 이동 평균을 계산

dataframe.ewm(alpha=0.5).mean()

            Stock_Price
2010-01-31            1
2010-02-28            2
2010-03-31            3
2010-04-30            4
2010-05-31            5
            Stock_Price
2010-01-31          NaN
2010-02-28          1.5
2010-03-31          2.5
2010-04-30          3.5
2010-05-31          4.5


,Stock_Price
2010-01-31,1.000000
2010-02-28,1.666667
2010-03-31,2.428571
2010-04-30,3.266667
2010-05-31,4.161290
